Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Python setup

We will install the following Python packages on the DSVM. 

**Notes**: 
1. These steps are suggested with the assumption that you are using the kernel `Python [conda env:py36]` for this lab. If you are using a different kernel, please update all references to a conda environment below to the kernel you are using to run this notebook.
2. If you cannot install these packages from Jupyter, please SSH (`ssh <username>@<ip.address.of.VM>`) into the DSVM from the command line and run these commands.

In [ ]:
%%bash 

conda install --name py36 seaborn
conda install --name py36 -f -c conda-forge jupyter_contrib_nbextensions
conda install --name py36 -f -c conda-forge jupyter_nbextensions_configurator

### Upgrade your azureml installation

Hopefully you won't have to do this. Especially if you are in one of our sessions, you shouldn't have to do this. 

If you still need to do this, follow the following step:

In [ ]:
%%bash

# you might have to uncomment the next line if you get a CommandNotFoundError
# . /anaconda/etc/profile.d/conda.sh

conda activate py36 # if you just run activate you get permission denied

In [ ]:
%%bash

pip install --upgrade pip
pip install --upgrade azureml-sdk[automl,notebooks] # I had to run it twice before it worked

# Creating an AML Workspace

In this example we create an Azure Machine Learning Workspace and initialize the notebook directory to easily use this workspace.  Typically we only need to run this once per notebook directory, and all other notebooks in this directory or any sub-directories will automatically use the settings indicate here.


## Prerequisites:

Before running this notebook, we run the `automl_setup` script described in README.md.


### Connect to your Azure Subscription

In order to use an AML Workspace, we first need access to an Azure subscription.

We first login to azure and follow prompts to authenticate. Once we authenticate from the browser, we should see a list of subscriptions you have access to.

In [ ]:
!az login

We can also list all our available subscriptions, using `-o table` to show results in a clean tabulated format.

In [ ]:
!az account list -o table

We need to check that your subscription is correct (the one where column called `IsDefault` is set to `True`). If not, we need to change the default subscription using `az account set -s <subscription-id>`.

In [ ]:
!az account set -s <subscription id>

Let's now confirm that our subscription changed.

In [ ]:
!az account show -o table

### Register Machine Learning Services Resource Provider

This step is required to use the Azure ML services backing the SDK.

In [ ]:
# register the new RP
!az provider register -n Microsoft.MachineLearningServices -o table

In [ ]:
# check the registration status
!az provider show -n Microsoft.MachineLearningServices -o table

### Check core SDK version number for validate your installation and for debugging purposes

In [ ]:
import azureml.core

print("SDK Version:", azureml.core.VERSION)

## Initialize an Azure ML Workspace
### What is an Azure ML Workspace and why do I need one?

> An AML Workspace is an Azure resource that organaizes and coordinates the actions of many other Azure resources to assist in executing and sharing machine learning workflows.  In particular, an AML Workspace coordinates storage, databases, and compute resources providing added functionality for machine learning experimentation, operationalization, and the monitoring of operationalized models.

To create or access an Azure ML Workspace, you will need to import the AML library and specify the following information:
- Your subscription id. Use *id* value from *az account show* output above. 
- The resource group name. Resource group organizes Azure resources and provides default region for the resources in the group. You can either specify a new one, in which case it gets created for your Workspace, or use an existing one or create a new one from [Azure portal](https://portal.azure.com)
- A name for your workspace. You can choose one.
- Your workspace region.

> For workspace region, we prefer you use `eastus2` or `eastus2euap` (only if you have access to EUAP) for most scenarios. Other supported regions include `westcentralus`, `southeastasia`, `westeurope`, `australiaeast`, although their support might lag behind `eastus2` and `eastus2euap`.

Let's begin by creating a new resource group in a given region. replace `<workspacename>` with the desired name of your workspace.

In [ ]:
! az group create -n wopauliamlws -l southcentralus

Let's suppose we have a python script that stores our subscription ID, resource group, workspace name and region. If no such file exists, we can create one by running the below script, which will output to a file called `./aml_config/mycreds.py`

In [ ]:
%%writefile ./aml_config/mycreds.py
subscription_id = '5be49961-ea44-42ec-8021-b728be90d58c'
resource_group ='wopauliaml'
workspace_name = 'wopauliamlws'
workspace_region = 'southcentralus' # eastus eastus2 eastus2euap

We can now simply run the above script.

In [ ]:
%run ./aml_config/mycreds.py

Alternatively, we could also create environment variables with this information. If no such variables exist, the snippet below will create them.

In [ ]:
%set_env SUBSCRIPTION_ID=$subscription_id
%set_env RESOURCE_GROUP=$resource_group
%set_env WORKSPACE_NAME=$workspace_name
%set_env WORKSPACE_REGION=$workspace_region

Let's make sure the environment variables were created for us.

In [ ]:
%env

## Creating a workspace

If we already have access to an AML workspace, we can skip this cell.  Otherwise, this cell will create an AML workspace under the subscription provided (assuming we have the correct permissions for the given subscription ID).

This will fail when:
1. The workspace already exists
2. You do not have permission to create a workspace in the resource group
3. You are not a subscription owner or contributor and no Azure ML workspaces have ever been created in this subscription

If workspace creation fails for any reason other than already existing, please work with your IT admin to provide you with the appropriate permissions or to provision the required resources.

**Note** The workspace creation can take several minutes.

In [ ]:
import os

subscription_id = os.environ.get("SUBSCRIPTION_ID")
resource_group = os.environ.get("RESOURCE_GROUP")
workspace_name = os.environ.get("WORKSPACE_NAME")
workspace_region = os.environ.get("WORKSPACE_REGION")

In [ ]:
# import the Workspace class and check the azureml SDK version
from azureml.core import Workspace, Experiment

ws = Workspace.create(name = workspace_name,
                      subscription_id = subscription_id,
                      resource_group = resource_group, 
                      location = workspace_region,
                      exist_ok = True)
ws.get_details()

Alternatively, we can also follow the instructions [here](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-configure-environment) to create an Azure ML Workspace from the Azure portal.

Once we have a workspace, we can create a config file for it using `write_config()`.

In [ ]:
try:
   ws = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)
   ws.write_config()
   print('Library configuration succeeded')
except:
   print('Workspace not found')

If we ran the above code successfully, we should have a new file called `aml_config/config.json`.

In [ ]:
!cat /home/sethmott/notebooks/AzureML/sethmott/aml_config/config.json

## Configuring your local environment

From now on, we can then load the workspace from this config file from any notebook in the current directory by simply running `from_config()`.

In [ ]:
# load workspace configuratio from ./aml_config/config.json file.
my_workspace = Workspace.from_config()
my_workspace.get_details()

## Create a folder to host all sample projects
Lastly, create a folder where all the sample projects will be hosted.

In [ ]:
!pwd

In [ ]:
import os

sample_projects_folder = '.'

if not os.path.isdir(sample_projects_folder):
    os.mkdir(sample_projects_folder)

> Create an experiment to track the runs in your workspace. A workspace can have muliple experiments; an experiment must belongn to a workspace.

In [ ]:
experiment_name = 'pred-maint-basic'

exp = Experiment(workspace=my_workspace, name=experiment_name)

## Success!
Great, you are ready to move on to the rest of the sample notebooks.

Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.